In [455]:
import numpy as np
import pandas as pd
import itertools
from sklearn.metrics import accuracy_score
import datetime
from sklearn.model_selection import train_test_split
from more_itertools import set_partitions


In [108]:
file_path = 'eye-color.data'

In [109]:
df = pd.read_csv(file_path, sep=',')
X_set = df.iloc[:,:-1]
y_set = df.iloc[:,-1]

In [110]:
X_set

,Eye,Hair,Height
0,Blue,Blonde,Tall
1,Blue,Brown,Medium
2,Brown,Brown,Medium
3,Green,Brown,Medium
4,Green,Brown,Tall
5,Brown,Brown,Low
6,Green,Blonde,Low
7,Blue,Brown,Medium


In [258]:
dataset= df
header = list(dataset)
dataset = dataset.values

In [112]:
dataset

array([['Blue', 'Blonde', 'Tall', 'C+'],
       ['Blue', 'Brown', 'Medium', 'C+'],
       ['Brown', 'Brown', 'Medium', 'C-'],
       ['Green', 'Brown', 'Medium', 'C-'],
       ['Green', 'Brown', 'Tall', 'C+'],
       ['Brown', 'Brown', 'Low', 'C-'],
       ['Green', 'Blonde', 'Low', 'C-'],
       ['Blue', 'Brown', 'Medium', 'C+']], dtype=object)

In [113]:
num_rows, num_cols = np.shape(dataset)
num_test_rows = num_rows
num_training_rows = num_rows

num_cols = num_cols -1
class_index = num_cols
feature_indices = list(range(class_index)) + list(range(class_index + 1, num_cols))

training = dataset[num_test_rows:]
test = dataset[:num_test_rows]
print('Number of training instances: ' + str(num_training_rows))
print('Number of test instances: ' + str(num_test_rows))

Number of training instances: 8
Number of test instances: 8


In [114]:
feature_indices

[0, 1, 2]

In [115]:

# Aggregate all the attribute values
attribute_values = [None] * num_cols
attribute_indices = [None] * num_cols
for attribute_index in feature_indices:
    attribute_values[attribute_index] = list(set([x[attribute_index] for x in dataset]))
    attribute_indices[attribute_index] = list(range(len(attribute_values[attribute_index])))

In [139]:
attribute_values

[['Green', 'Blue', 'Brown'], ['Blonde', 'Brown'], ['Medium', 'Tall', 'Low']]

In [177]:
feature_subset

[0, 1, 2]

In [116]:
attribute_values

[['Green', 'Blue', 'Brown'], ['Blonde', 'Brown'], ['Medium', 'Tall', 'Low']]

In [117]:
attribute_indices

[[0, 1, 2], [0, 1], [0, 1, 2]]

In [118]:
def create_attribute_subsets(subset, attribute_index):
    attribute_subsets = {}
    for sample in subset:
        if sample[attribute_index] in attribute_subsets:
            attribute_subsets[sample[attribute_index]].append(sample)
        else:
            attribute_subsets[sample[attribute_index]] = [sample]
    return attribute_subsets

In [119]:
#def build_tree(subset, unused_attributes, feature_subset_size):

subset, unused_attributes, feature_subset_size = test, feature_indices, len(feature_indices)

# If all elements of subset are in the same class we can make this a leaf node
if len(set([x[class_index] for x in subset])) == 1:
    tree = subset[0][class_index]

if len(unused_attributes) == 0:
    # Get most common class
    tree = get_most_common_class(subset)

# Apply the random forest technique of only considering a subset of features here
# but only if we have enough features to begin with.
feature_subset = unused_attributes
if len(feature_subset) > feature_subset_size:
    feature_subset = random.sample(feature_subset, feature_subset_size)

In [120]:
set(attribute_values[0])

{'Blue', 'Brown', 'Green'}

In [249]:
best_split_gini = 10
attribute_index = None
best_split_value = None
best_split_groups  = None
gini_X = single_gini_index(df)
for attribute_index in feature_subset:
    partitions = list(set_partitions(attribute_values[attribute_index], 2))
    for part in partitions:
        left_split, right_split = build_split(df, attribute_index, part)
        gini_XA =  multi_gini_index(left_split, right_split)
       ob gini_A = gini_X - gini_XA
        print(gini_A)
        if gini_XA < best_split_gini:
            best_split_gini = gini_XA
            best_split_column  = attribute_index
            best_split_value = part
            best_split_groups = left_split, right_split
return_dict = {'column_name': best_split_column,'dsplit_value':best_split_value,
             'gini':best_split_gini, 'groups': best_split_groups}

0.033333333333333326
0.16666666666666669
0.3000000000000001
0.0
0.0
0.16666666666666669
0.16666666666666669


In [228]:
best_split_value

[['Blue'], ['Green', 'Brown']]

In [230]:
best_split_groups[0]

,Eye,Hair,Height,class
0,Blue,Blonde,Tall,C+
1,Blue,Brown,Medium,C+
7,Blue,Brown,Medium,C+


In [135]:
df.loc[df[header[0]].isin(['Brown', 'Green'])]

,Eye,Hair,Height,class
2,Brown,Brown,Medium,C-
3,Green,Brown,Medium,C-
4,Green,Brown,Tall,C+
5,Brown,Brown,Low,C-
6,Green,Blonde,Low,C-


In [414]:
def build_split(data,column_to_split,split_values):
    '''build 2 groups of data by splitting data on the column_to_split 
       at the split_value'''

    left_split = data.loc[data[header[column_to_split]].isin(split_values[0])]
    right_split = data.loc[data[header[column_to_split]].isin(split_values[1])]
    
    return left_split,right_split

In [163]:
def multi_gini_index(group1,group2):
    '''Calculate Gini Impurity, func expects to be passed 
       the 2 groups of data that are the result of a split'''
    class_proportions_group1 = group1['class'].value_counts(normalize=True)    
    class_proportions_group2 = group2['class'].value_counts(normalize=True)    

    instance_proportion_group1 = len(group1)/(len(group1)+len(group2))
    instance_proportion_group2 = len(group2)/(len(group1)+len(group2))

    gini1 = (1 - class_proportions_group1.pow(2).sum())*(instance_proportion_group1)
    gini2 = (1 - class_proportions_group2.pow(2).sum())*(instance_proportion_group2)
    gini = gini1+gini2

    return gini

def single_gini_index(group):
    '''Calculate Gini Impurity of a single group'''
    class_proportions = group['class'].value_counts(normalize=True)    

    gini = (1 - class_proportions.pow(2).sum())
  
    return gini

In [232]:
best_split_groups[1]

,Eye,Hair,Height,class
2,Brown,Brown,Medium,C-
3,Green,Brown,Medium,C-
4,Green,Brown,Tall,C+
5,Brown,Brown,Low,C-
6,Green,Blonde,Low,C-


In [256]:
find_best_split_point(best_split_groups[1], feature_subset)

['Green', 'Brown']
[[['Green'], ['Brown']]]
0.26666666666666666
['Blonde', 'Brown']
[[['Blonde'], ['Brown']]]
0.30000000000000004
['Medium', 'Tall', 'Low']
[[['Medium'], ['Tall', 'Low']], [['Medium', 'Tall'], ['Low']], [['Tall'], ['Medium', 'Low']]]
0.26666666666666666
0.26666666666666666
0.0


{'column_name': 2,
 'dsplit_value': [['Tall'], ['Medium', 'Low']],
 'gini': 0.31999999999999984,
 'groups': (     Eye   Hair Height class
  4  Green  Brown   Tall    C+,
       Eye    Hair  Height class
  2  Brown   Brown  Medium    C-
  3  Green   Brown  Medium    C-
  5  Brown   Brown     Low    C-
  6  Green  Blonde     Low    C-)}

In [463]:
types=list(df.dtypes)

In [464]:
types[0] == 'O'

True

In [415]:
def find_best_split_point(passed_data, feature_subset):
    '''find best split point iterating over range of values returned from the 
    get_range_to_split_on function and return a dictionary which functions as a node '''

    best_split_gini = 10
    attribute_index = None
    best_split_value = None
    best_split_groups  = None
    best_split_column = None
    gini_X = single_gini_index(passed_data)
    for attribute_index in feature_subset:
        
        attribute_values = list(set([x[attribute_index] for x in passed_data.values]))
        if len(attribute_values) == 1:
            gini_XA = single_gini_index(passed_data)
            if gini_XA < best_split_gini:
                best_split_gini = gini_XA
                best_split_column  = attribute_index
                best_split_value = attribute_values
                best_split_groups = passed_data, pd.DataFrame(columns = passed_data.columns)
        else:    
            partitions = list(set_partitions(attribute_values, 2))
            for part in partitions:
                if len(part[1]) < len(part[0]):
                    part = [part[1], part[0]]    
                left_split, right_split = build_split(passed_data, attribute_index, part)
                gini_XA =  multi_gini_index(left_split, right_split)
                gini_A = gini_X - gini_XA
                if gini_XA < best_split_gini:
                    best_split_gini = gini_XA
                    best_split_column  = attribute_index
                    best_split_value = part
                    best_split_groups = left_split, right_split
    return {'column_name': best_split_column,'dsplit_value':best_split_value,
                 'gini':best_split_gini, 'groups': best_split_groups}

In [456]:
dd = df.copy()

In [457]:
nn = find_best_split_point(df, [0,1])

In [458]:
nn

{'column_name': 0,
 'dsplit_value': [['Blue'], ['Green', 'Brown']],
 'gini': 0.1999999999999999,
 'groups': (    Eye    Hair  Height class
  0  Blue  Blonde    Tall    C+
  1  Blue   Brown  Medium    C+
  7  Blue   Brown  Medium    C+,
       Eye    Hair  Height class
  2  Brown   Brown  Medium    C-
  3  Green   Brown  Medium    C-
  4  Green   Brown    Tall    C+
  5  Brown   Brown     Low    C-
  6  Green  Blonde     Low    C-)}

In [444]:
def recursive_splitter(node,random_features):
    '''this function recursively splits the data starting with the root node which its passed
    untill the groups are homogenous or further splits result in empty nodes'''
    left_group,right_group = node['groups']
    #delete the groups entry in original node
    del node['groups']
    #check if the groups of the node are empty
    if left_group.empty or right_group.empty:
        #combine as we will use original to predict
        combined = pd.concat([left_group,right_group])
        predicted_class = combined['class'].value_counts().index[0]
        node['left']=node['right']=predicted_class
        return [predicted_class]
    #check if the groups of the node are homogenous otherwise call recursive_spltter again
    if single_gini_index(left_group) == 0:
        predicted_class = left_group['class'].value_counts().index[0]
        node['left'] = predicted_class
    else:
        node['left'] = find_best_split_point(left_group,random_features)
        curr_node = recursive_splitter(node['left'],random_features)
        if type(curr_node) == list:
            node['left'] = curr_node[0]

    if single_gini_index(right_group) == 0:
        predicted_class = right_group['class'].value_counts().index[0]
        node['right'] = predicted_class
    else:
        node['right'] = find_best_split_point(right_group,random_features)
        curr_node = recursive_splitter(node['right'],random_features)
        if type(curr_node) == list:
            node['right'] = curr_node[0]        
    return node

In [402]:
df

,Eye,Hair,Height,class
0,Blue,Blonde,Tall,C+
1,Blue,Brown,Medium,C+
2,Brown,Brown,Medium,C-
3,Green,Brown,Medium,C-
4,Green,Brown,Tall,C+
5,Brown,Brown,Low,C-
6,Green,Blonde,Low,C-
7,Blue,Brown,Medium,C+


In [428]:
type(a) == dict

True

In [459]:
a = recursive_splitter(nn, [0,1])

In [460]:
a

{'column_name': 0,
 'dsplit_value': [['Blue'], ['Green', 'Brown']],
 'gini': 0.1999999999999999,
 'left': 'C+',
 'right': {'column_name': 0,
  'dsplit_value': [['Green'], ['Brown']],
  'gini': 0.26666666666666666,
  'left': {'column_name': 1,
   'dsplit_value': [['Blonde'], ['Brown']],
   'gini': 0.3333333333333333,
   'left': 'C-',
   'right': 'C+'},
  'right': 'C-'}}

In [453]:
arr = ['Green', 'Brown', 'Tall']

In [ ]:
make_prediction_tree

In [450]:
arr[0] in a['dsplit_value'][1]

True

In [454]:
make_prediction_tree(arr, a)

'C+'

In [451]:
def make_prediction_tree(data_row,root_node):
    '''recursively traverse the tree from root to leaf turning left if feature value
    to test is less than dsplit_value or right otherwise until we reach a leaf node'''
    
    #check if feature of data_row is less than dsplit_value else move to right branch
    if data_row[root_node['column_name']] in root_node['dsplit_value'][0]:
        #check if at a branch or a leaf if branch recursively call predict else return leaf prediction
        if type(root_node['left']) is dict:
            return make_prediction_tree(data_row,root_node['left'])
        else:
            return root_node['left']
    else:
        if type(root_node['right']) is dict:
            return make_prediction_tree(data_row,root_node['right'])
        else:
            return root_node['right']  

In [ ]:


min_index, min_entropy = 0, sys.maxsize
for attribute_index in feature_subset:
    # Create subsets for that given attribute
    attribute_subsets = create_attribute_subsets(subset, attribute_index)

    # Calculate entropy of the generated subsets and weight them according to their size
    entropy = 0
    for attribute_subset in attribute_subsets.values():
        entropy += float(len(attribute_subset) / len(subset)) * calculate_entropy(attribute_subset)

    # Keep track of the split attribute with minimum entropy
    if entropy < min_entropy:
        min_index, min_entropy = attribute_index, entropy

# Copy list otherwise our passed feature_indices array will be modified.
unused_attributes = list(unused_attributes)
unused_attributes.remove(min_index)

# Reconstruct subsets of best attribute and use them to build subtrees recursively
# As a measure of feature importance we save the feature importance (information gain) here
attribute_subsets = create_attribute_subsets(subset, min_index)
tree = {'attribute_index': min_index, 'subtrees': {}, 'information_gain': calculate_entropy(subset) - min_entropy}

# Make sure to add for all values that exist in the training dataset labels
most_common_class = None
for attribute_value in attribute_values[min_index]:
    if attribute_value in attribute_subsets and len(attribute_subsets[attribute_value]):
        tree['subtrees'][attribute_value] = build_tree(attribute_subsets[attribute_value], unused_attributes,
                                                       feature_subset_size)
    else:
        if not most_common_class:
            most_common_class = get_most_common_class(subset)
        tree['subtrees'][attribute_value] = most_common_class

#return tree

In [ ]:
def find_best_split_point(passed_data, random_features):
    '''find best split point iterating over range of values returned from the 
    get_range_to_split_on function and return a dictionary which functions as a node '''

In [65]:

def create_class_counts(subset):
    class_counts = {}
    for sample in subset:
        if sample[class_index] in class_counts:
            class_counts[sample[class_index]] += 1
        else:
            class_counts[sample[class_index]] = 1
    return class_counts


def get_most_common_class(subset):
    max_class_count, most_common_class = -1, ''
    class_counts = create_class_counts(subset)
    for the_class, class_count in class_counts.items():
        if class_count > max_class_count:
            max_class_count, most_common_class = class_count, the_class
    return most_common_class


def calculate_entropy(subset):
    class_counts = create_class_counts(subset)
    subset_entropy = 0
    for class_count in class_counts.values():
        p_x = float(class_count) / len(subset)
        subset_entropy -= p_x * math.log(p_x)
    return subset_entropy


In [ ]:
def find_best_split_point(passed_data, random_features):
    '''find best split point iterating over range of values returned from the 
    get_range_to_split_on function and return a dictionary which functions as a node '''

    
     #intialise values for best split point
    best_split_column = 'name'
    best_split_value = 0
    best_split_gini = 10
    best_split_groups = None
    
    #iterate over columns and rows searching for best split point
    for col_name in random_features:
        # for split_value in splitpoints_dict[col_name]:
        for index,row in passed_data.iterrows():
            left_split, right_split = build_split(passed_data,col_name,row[col_name])
            gini_score = multi_gini_index(left_split, right_split)

            if gini_score < best_split_gini:
                best_split_gini = gini_score
                best_split_column = col_name
                best_split_value = row[col_name]
                best_split_groups = left_split, right_split
    
    
    #print(best_split_column,best_split_gini)
    return {'column_name': best_split_column,'dsplit_value':best_split_value,
             'gini':best_split_gini, 'groups': best_split_groups}

def build_split(data,column_to_split,split_value):
    '''build 2 groups of data by splitting data on the column_to_split 
       at the split_value'''
    left_split = data[data[column_to_split]<split_value]
    right_split = data[data[column_to_split]>=split_value]
    
    return left_split,right_split

    
def multi_gini_index(group1,group2):
    '''Calculate Gini Impurity, func expects to be passed 
       the 2 groups of data that are the result of a split'''
    class_proportions_group1 = group1['class'].value_counts(normalize=True)    
    class_proportions_group2 = group2['class'].value_counts(normalize=True)    

    instance_proportion_group1 = len(group1)/(len(group1)+len(group2))
    instance_proportion_group2 = len(group2)/(len(group1)+len(group2))

    gini1 = (1 - class_proportions_group1.pow(2).sum())*(instance_proportion_group1)
    gini2 = (1 - class_proportions_group2.pow(2).sum())*(instance_proportion_group2)
    gini = gini1+gini2

    return gini

def single_gini_index(group):
    '''Calculate Gini Impurity of a single group'''
    class_proportions = group['class'].value_counts(normalize=True)    

    gini = (1 - class_proportions.pow(2).sum())
  
    return gini

In [18]:
def plant_tree(train_data, number_features=3,num_bins=10):
    ''' start the process of recursion on the training data and let the tree
    grow to its max depth using subset of random features'''
      
    #get column names minus class
    #choose random set of features from column names
    data_column_names = list(train_data.columns)
    data_column_names.remove('class')
    random_features = random.sample(data_column_names,number_features)

    root_node = find_best_split_point(train_data,random_features)
    recursive_spltter(root_node,random_features)
    return root_node,random_features

In [31]:
import numpy as np


class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None


class DecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_classes_ = len(set(y))
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

    def _best_split(self, X, y):
        m = y.size
        if m <= 1:
            return None, None
        num_parent = [np.sum(y == c) for c in range(self.n_classes_)]
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None
        for idx in range(self.n_features_):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes_
            num_right = num_parent.copy()
            for i in range(1, m):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes_)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_)
                )
                gini = (i * gini_left + (m - i) * gini_right) / m
                if thresholds[i] == thresholds[i - 1]:
                    continue
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        return best_idx, best_thr

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(predicted_class=predicted_class)
        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class


if __name__ == "__main__":
    import sys
    from sklearn.datasets import load_iris

    dataset = load_iris()
    X, y = dataset.data, dataset.target  # pylint: disable=no-member
    clf = DecisionTreeClassifier(max_depth=1)
    clf.fit(X, y)
    print(clf.predict([[6.9, 3.1, 5.1, 2.3]]))

[1]


In [24]:
print(clf.tree_.left)

In [25]:
dataset

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [19]:
def recursive_spltter(node,random_features):
    '''this function recursively splits the data starting with the root node which its passed
    untill the groups are homogenous or further splits result in empty nodes'''
    #from IPython.core.debugger import Tracer; Tracer()() 
    #retrieve two groups from the passed which is root or a recursive call on itself
    left_group,right_group = node['groups']
    #delete the groups entry in original node
    del node['groups']
    #check if the groups of the node are empty
    if left_group.empty or right_group.empty:
        #combine as we will use original to predict
        combined = pd.concat([left_group,right_group])
        predicted_class = combined['class'].value_counts().index[0]
        node['left']=node['right']=predicted_class
        return
    #check if the groups of the node are homogenous otherwise call recursive_spltter again
    if single_gini_index(left_group) == 0:
        predicted_class = left_group['class'].value_counts().index[0]
        node['left'] = predicted_class
    else :
        node['left'] = find_best_split_point(left_group,random_features)
        recursive_spltter(node['left'],random_features)

    if single_gini_index(right_group) == 0:
        predicted_class = right_group['class'].value_counts().index[0]
        node['right'] = predicted_class
    
    else:
        node['right'] = find_best_split_point(right_group,random_features)
        recursive_spltter(node['right'],random_features)

def find_best_split_point(passed_data, random_features):
    '''find best split point iterating over range of values returned from the 
    get_range_to_split_on function and return a dictionary which functions as a node '''

    
     #intialise values for best split point
    best_split_column = 'name'
    best_split_value = 0
    best_split_gini = 10
    best_split_groups = None
    
    #iterate over columns and rows searching for best split point
    for col_name in random_features:
        # for split_value in splitpoints_dict[col_name]:
        for index,row in passed_data.iterrows():
            left_split, right_split = build_split(passed_data,col_name,row[col_name])
            gini_score = multi_gini_index(left_split, right_split)

            if gini_score < best_split_gini:
                best_split_gini = gini_score
                best_split_column = col_name
                best_split_value = row[col_name]
                best_split_groups = left_split, right_split
    
    
    #print(best_split_column,best_split_gini)
    return {'column_name': best_split_column,'dsplit_value':best_split_value,
             'gini':best_split_gini, 'groups': best_split_groups}

def build_split(data,column_to_split,split_value):
    '''build 2 groups of data by splitting data on the column_to_split 
       at the split_value'''
    left_split = data[data[column_to_split]<split_value]
    right_split = data[data[column_to_split]>=split_value]
    
    return left_split,right_split

    
def multi_gini_index(group1,group2):
    '''Calculate Gini Impurity, func expects to be passed 
       the 2 groups of data that are the result of a split'''
    class_proportions_group1 = group1['class'].value_counts(normalize=True)    
    class_proportions_group2 = group2['class'].value_counts(normalize=True)    

    instance_proportion_group1 = len(group1)/(len(group1)+len(group2))
    instance_proportion_group2 = len(group2)/(len(group1)+len(group2))

    gini1 = (1 - class_proportions_group1.pow(2).sum())*(instance_proportion_group1)
    gini2 = (1 - class_proportions_group2.pow(2).sum())*(instance_proportion_group2)
    gini = gini1+gini2

    return gini

def single_gini_index(group):
    '''Calculate Gini Impurity of a single group'''
    class_proportions = group['class'].value_counts(normalize=True)    

    gini = (1 - class_proportions.pow(2).sum())
  
    return gini

In [ ]:




def create_attribute_subsets(subset, attribute_index):
    attribute_subsets = {}
    for sample in subset:
        if sample[attribute_index] in attribute_subsets:
            attribute_subsets[sample[attribute_index]].append(sample)
        else:
            attribute_subsets[sample[attribute_index]] = [sample]
    return attribute_subsets


def create_class_counts(subset):
    class_counts = {}
    for sample in subset:
        if sample[class_index] in class_counts:
            class_counts[sample[class_index]] += 1
        else:
            class_counts[sample[class_index]] = 1
    return class_counts


def get_most_common_class(subset):
    max_class_count, most_common_class = -1, ''
    class_counts = create_class_counts(subset)
    for the_class, class_count in class_counts.items():
        if class_count > max_class_count:
            max_class_count, most_common_class = class_count, the_class
    return most_common_class


def calculate_entropy(subset):
    class_counts = create_class_counts(subset)
    subset_entropy = 0
    for class_count in class_counts.values():
        p_x = float(class_count) / len(subset)
        subset_entropy -= p_x * math.log(p_x)
    return subset_entropy


# Aggregate all the attribute values
attribute_values = [None] * num_cols
for attribute_index in feature_indices:
    attribute_values[attribute_index] = list(set([x[attribute_index] for x in dataset]))


def build_tree(subset, unused_attributes, feature_subset_size):
    # If all elements of subset are in the same class we can make this a leaf node
    if len(set([x[class_index] for x in subset])) == 1:
        return subset[0][class_index]

    if len(unused_attributes) == 0:
        # Get most common class
        return get_most_common_class(subset)

    # Apply the random forest technique of only considering a subset of features here
    # but only if we have enough features to begin with.
    feature_subset = unused_attributes
    if len(feature_subset) > feature_subset_size:
        feature_subset = random.sample(feature_subset, feature_subset_size)

    min_index, min_entropy = 0, sys.maxsize
    for attribute_index in feature_subset:
        # Create subsets for that given attribute
        attribute_subsets = create_attribute_subsets(subset, attribute_index)

        # Calculate entropy of the generated subsets and weight them according to their size
        entropy = 0
        for attribute_subset in attribute_subsets.values():
            entropy += float(len(attribute_subset) / len(subset)) * calculate_entropy(attribute_subset)

        # Keep track of the split attribute with minimum entropy
        if entropy < min_entropy:
            min_index, min_entropy = attribute_index, entropy

    # Copy list otherwise our passed feature_indices array will be modified.
    unused_attributes = list(unused_attributes)
    unused_attributes.remove(min_index)

    # Reconstruct subsets of best attribute and use them to build subtrees recursively
    # As a measure of feature importance we save the feature importance (information gain) here
    attribute_subsets = create_attribute_subsets(subset, min_index)
    tree = {'attribute_index': min_index, 'subtrees': {}, 'information_gain': calculate_entropy(subset) - min_entropy}

    # Make sure to add for all values that exist in the training dataset labels
    most_common_class = None
    for attribute_value in attribute_values[min_index]:
        if attribute_value in attribute_subsets and len(attribute_subsets[attribute_value]):
            tree['subtrees'][attribute_value] = build_tree(attribute_subsets[attribute_value], unused_attributes,
                                                           feature_subset_size)
        else:
            if not most_common_class:
                most_common_class = get_most_common_class(subset)
            tree['subtrees'][attribute_value] = most_common_class

    return tree


def classify(example, tree):
    if isinstance(tree, str):
        return tree
    else:
        return classify(example, tree['subtrees'][example[tree['attribute_index']]])

# Counts predictions of all trees in the forest and selects the one with the highest count
def majority_vote(example, trees):
    votes = {}
    for tree in trees:
        prediction = classify(example, tree)
        if prediction in votes:
            votes[prediction] += 1
        else:
            votes[prediction] = 1
    majority_prediction, max_vote = None, 0
    for prediction, num_votes in votes.items():
        if num_votes > max_vote:
            majority_prediction, max_vote = prediction, num_votes
    return majority_prediction


# Translate attribute indices into human readable columns
def pretty_tree(tree):
    if isinstance(tree, str):
        return tree
    else:
        pretty_subtrees = {}
        for attribute_value, subtree in tree['subtrees'].items():
            pretty_subtrees[attribute_value] = pretty_tree(subtree)
        return { 'attribute_index' : header[tree['attribute_index']], 'subtrees' : pretty_subtrees}


# Estimate importance of a feature by calculating the average information gain
def get_feature_importance(tree, feature_importance):
    if not isinstance(tree, str):
        if header[tree['attribute_index']] in feature_importance:
            feature_importance[header[tree['attribute_index']]].append(tree['information_gain'])
        else:
            feature_importance[header[tree['attribute_index']]] = [tree['information_gain']]
        for subtree in tree['subtrees'].values():
            get_feature_importance(subtree, feature_importance)


# Feature importance of a forest is calculated by averaging the feature importances of the individual trees
def get_feature_importance_for_forest(trees):
    feature_importance = {}
    for tree in trees:
        get_feature_importance(tree, feature_importance)

    # Average the information gains
    for feature_name, importance in feature_importance.items():
        feature_importance[feature_name] = float(sum(importance)) / len(importance)

    # Finally sort feature importance in descending order
    return sorted(feature_importance.items(), key=operator.itemgetter(1), reverse=True)


# Use a single tree to test our ID3 algorithm.
# We pass sys.maxsize so no random feature sampling is used.
the_tree = build_tree(training, feature_indices, sys.maxsize)
#print(pretty_tree(the_tree))

num_classified_correctly = sum([classify(example, the_tree) ==
                                example[class_index] for example in test])
print('Single tree: Accuracy on test dataset: {:.2f}%'.format(float(100 * num_classified_correctly) / num_test_rows))


# Perform bootstrap aggregating: Build trees from multiple sample sets
trees = []
for i in range(args.num_trees):
    bootstrap_sample_set = [random.choice(training) for j in range(len(training))]
    trees.append(build_tree(bootstrap_sample_set, feature_indices, sys.maxsize))

num_classified_correctly = sum([majority_vote(example, trees) ==
                                example[class_index] for example in test])
print('Bagging: Accuracy on test dataset: {:.2f}%'.format(float(100 * num_classified_correctly) / num_test_rows))

# Random forest: Additionally select subset of features
trees = []
for i in range(args.num_trees):
    bootstrap_sample_set = [random.choice(training) for j in range(len(training))]
    trees.append(build_tree(bootstrap_sample_set, feature_indices, args.feature_subset_size))

num_classified_correctly = sum([majority_vote(example, trees) ==
                                example[class_index] for example in test])
print('Random forest: Accuracy on test dataset: {:.2f}%'.format(float(100 * num_classified_correctly) / num_test_rows))

# Print out feature importances
for rank, feature_importance in enumerate(get_feature_importance_for_forest(trees)):
    print(str(rank + 1) + ': ' + str(feature_importance[0]) + ' {:.3}'.format(feature_importance[1]))


In [7]:
# Reading and cleaning the data
titanic = pd.read_csv("train.csv")
target = titanic["Survived"]
features = titanic.loc[:, ["Pclass", "Sex", "Embarked"]]
features.Embarked.fillna(features.Embarked.mode()[0], inplace=True)
features.Pclass = features.Pclass.map({1: "1st", 2: "2nd", 3: "3rd"})

In [10]:
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [8]:
# Main class
class Node:
    def __init__(self, features, target):
        self.left = None 
        self.right = None
        self.features = features
        self.target = target 
        self.feature_types = self.select_dtype()
        
    # Identiy feature types
    def select_dtype(self):
        feature_types = []
        for item in self.features.columns:
            if self.features[item].dtype != "O":
                feature_types.append((item, "numerical")) 
            else:
                if self.features[item].nunique() <= 2:
                    feature_types.append((item, "binary"))
                else:
                    feature_types.append((item, "multiclass"))
        return feature_types
    
    # Calculate gini impurity for each feature at a node
    # Need to provide a default value as a fallback 
    @staticmethod
    def gini_impurity_total(a=0, b=0, c=0, d=0):
        total_elements = a + b + c + d
        gini_1 = 1 - np.square(a/(a+b)) - np.square(b/(a+b))
        gini_2 = 1 - np.square(c/(c+d)) - np.square(d/(c+d))
        total_gini = ((a+b)/total_elements) * gini_1 + ((c+d)/total_elements) * gini_2
        return total_gini 
    
    @staticmethod
    def gini_impurity(a=0, b=0):
        return 1 - np.square(a/(a+b)) - np.square(b/(a+b))
    
    # Calculate gini for all feature combinations in categorical features
    def calculate_gini(self, feature):
        gini_node = []
        combinations = []
        
        for i in range(1, self.features[feature].nunique()):
            combinations = combinations + list(itertools.combinations(self.features[feature].unique(), i))
            
        for item in combinations:
            t1 = self.target[self.features[feature].isin(item)] 
            t2 = self.target[~self.features[feature].isin(item)]
            args = t1.value_counts().tolist() + t2.value_counts().tolist()
            gini_node.append(Node.gini_impurity_total(*args))
        
        return gini_node, combinations # Return all the values
     # Get the best gini values for each feature  
    def evaluate_node(self):
        gini_values = []
        for feature in self.features.columns:
            calculated_gini, combinations = self.calculate_gini(feature)
            best_combination = combinations[np.argmin(calculated_gini)]
            gini_values.append((feature, best_combination, np.min(calculated_gini)))
        return gini_values
                
    # Inserting a new node based on the decision criteria
    def insert_node(self):
        gini_values = self.evaluate_node()
        values = [item[2] for item in gini_values]
        node_gini =  Node.gini_impurity(*self.target.value_counts().tolist())
        
         # Terminate the branch if current gini is better or no features to split
        if node_gini < np.min(values): 
            print("terminating the branch")
            self.left = None
            self.right = None 
        else:
            best_feature = gini_values[np.argmin(values)][0]
            best_combination = gini_values[np.argmin(values)][1]
            print(f"Creating a new branch using {best_feature} and {best_combination}")

            left_features = self.features[self.features[best_feature].isin(best_combination)]
            left_features.drop([best_feature], axis=1, inplace=True)
            left_target = self.target[self.features[best_feature].isin(best_combination)]
            if list(left_features.columns) == []:
                self.left = None 
                self.right = None
            else:
                self.left = Node(left_features, left_target)
                self.left.insert_node()

            right_features = self.features[~self.features[best_feature].isin(best_combination)]
            right_target = self.target[~self.features[best_feature].isin(best_combination)]
            right_features.drop([best_feature], axis=1, inplace=True)
            if list(right_features.columns) == []:
                self.left = None 
                self.right = None
            else:
                self.right = Node(right_features, right_target)
                self.right.insert_node()

# Creating the root node with the full dataset 
root = Node(features, target)


In [9]:
root.insert_node()

Creating a new branch using Sex and ('male',)
Creating a new branch using Pclass and ('1st',)
Creating a new branch using Embarked and ('Q',)
Creating a new branch using Embarked and ('C',)
Creating a new branch using Pclass and ('3rd',)
Creating a new branch using Embarked and ('S',)
Creating a new branch using Embarked and ('Q',)


c:\users\nadzi\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
